In [1]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import unit_square

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))
mesh.GetBoundaries()

['bottom', 'right', 'top', 'left']

In [3]:
fes = H1(mesh, order=2, dirichlet='left|right')

In [4]:
fs2 = H1(mesh, order=2)

In [5]:
fes.ndof, fs2.ndof

(125, 125)

In [6]:
print("fs2 free dofs: ", fs2.FreeDofs())
print("fes free dofs: ", fes.FreeDofs())

fs2 free dofs:  0: 11111111111111111111111111111111111111111111111111
50: 11111111111111111111111111111111111111111111111111
100: 1111111111111111111111111
fes free dofs:  0: 00001111000011110000111111111111111111010101101111
50: 11111110101110111111111111111011010111111111111111
100: 1111111111111111111111111


In [7]:
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI(grad(u)*grad(v))
a.Assemble()


In [8]:
g = sin(y)

In [9]:
gfu = GridFunction(fes)
gfu.Set(g,BND)
Draw(gfu)

In [10]:
f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()

r = f.vec.CreateVector()
r.data = f.vec - a.mat * gfu.vec

In [11]:
c = Preconditioner(a, "direct")
c.Update()
BVP(bf=a, lf=f, gf=gfu, pre=c).Do()
Redraw()